# Langchain Tutorial: Prompt Engineering
* [Article](https://www.pinecone.io/learn/langchain-prompt-templates/)
* [Notebook](https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/01-langchain-prompt-templates.ipynb)

# Pre-Reqs

## Install Libraries

In [1]:
!pip install -Uqq langchain openai

## Load Secrets

In [3]:
%run utils/load-secrets.ipynb

# Prompt Engineering

## Hardcoded Prompt

In [5]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [12]:
from langchain import OpenAI

In [13]:
llm = OpenAI(
    model='text-davinci-003',
    openai_api_key=secrets['openai']
)

In [14]:
result = llm(prompt)

In [15]:
result

" Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library."

## Prompt Template

In [20]:
from langchain import PromptTemplate, LLMChain

In [21]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: {context}

Question: {question}

Answer: """

In [22]:
prompt_template = PromptTemplate(
    template = template,
    input_variables = ['context','question']
)

In [23]:
llm_chain = LLMChain(
    prompt=prompt_template,
    llm=llm
)

In [31]:
context = """Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library."""

In [26]:
question1 = 'Which libraries and model providers offer LLMs?'

In [27]:
question2 = 'How can LLMs be accessed?'

In [28]:
question3 = 'Who is Batman?'

In [38]:
print(
    prompt_template.format(
        context=context,
        question=question1
    )
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [36]:
result_q1 = llm_chain.run(context=context, question=question1)
result_q1

" Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library."

In [34]:
result_q2 = llm_chain.run(context=context, question=question2)
result_q2

" LLMs can be accessed via Hugging Face's `transformers` library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library."

In [35]:
result_q3 = llm_chain.run(context=context, question=question3)
result_q3

" I don't know."

## Few Shot Prompt Template

In [42]:
from langchain import FewShotPromptTemplate

In [43]:
?FewShotPromptTemplate

Init signature:
FewShotPromptTemplate(
    *,
    input_variables: List[str],
    output_parser: Optional[langchain.schema.BaseOutputParser] = None,
    partial_variables: Mapping[str, Union[str, Callable[[], str]]] = None,
    examples: Optional[List[dict]] = None,
    example_selector: Optional[langchain.prompts.example_selector.base.BaseExampleSelector] = None,
    example_prompt: langchain.prompts.prompt.PromptTemplate,
    suffix: str,
    example_separator: str = '\n\n',
    prefix: str = '',
    template_format: str = 'f-string',
    validate_template: bool = True,
) -> None
Docstring:      Prompt template that contains few shot examples.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/.local/lib/python3.10/site-packages/langchain/prompts/few_shot.py
Type:           ModelMetaclass
Subclasses:     


In [49]:
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:"""

In [60]:
suffix = """
User: {question}
AI: """

In [66]:
examples = [
    { 'question': 'How are you?', 'answer': 'I can\'t complain but sometimes I still do.'},
    { 'question': 'What time is it?', 'answer': 'It\'s time to get a watch.'},
]

In [67]:
example_template = """
User: {question}
AI: {answer}
"""

In [68]:
example_prompt = PromptTemplate(
    template=example_template,
    input_variables=['question','answer'],
)

In [69]:
template_fs = FewShotPromptTemplate(
    prefix=prefix,
    suffix=suffix,
    example_prompt=example_prompt,
    examples=examples,
    input_variables=['question'],
)

In [70]:
llm.temperature = 1.0

In [71]:
print(
    template_fs.format(question='What is the meaning of life?')
)

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:


User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What is the meaning of life?
AI: 


In [72]:
llm(template_fs.format(question='What is the meaning of life?'))

" 42. Doesn't that answer all your questions?"

## Length Based Example Selector

In [74]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [78]:
examples_long = [
    {
        "question": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "question": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "question": "What is the meaning of life?",
        "answer": "42"
    }, {
        "question": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "question": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "question": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "question": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [79]:
?LengthBasedExampleSelector

Init signature:
LengthBasedExampleSelector(
    *,
    examples: List[dict],
    example_prompt: langchain.prompts.prompt.PromptTemplate,
    get_text_length: Callable[[str], int] = <function _get_length_based at 0x7faa8ece1480>,
    max_length: int = 2048,
    example_text_lengths: List[int] = [],
) -> None
Docstring:      Select examples based on length.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/.local/lib/python3.10/site-packages/langchain/prompts/example_selector/length_based.py
Type:           ModelMetaclass
Subclasses:     


In [80]:
example_selector = LengthBasedExampleSelector(
    examples=examples_long,
    example_prompt=example_prompt,
    max_length=50
)

In [85]:
fs_es_template = FewShotPromptTemplate(
    prefix=prefix,
    suffix=suffix,
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=['question'],
    example_separator='\n'
)

In [86]:
print(fs_es_template.format(question="Who are you?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:

User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: Who are you?
AI: 


In [87]:
print(fs_es_template.format(question="What is the date in which you first appeared in this world?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:

User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the date in which you first appeared in this world?
AI: 
